In [ ]:
from helpers import calculate_vector_field_eigenpairs
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(10)

In [ ]:
import numpy as np
xi = np.random.uniform(-np.sqrt(3), np.sqrt(3), randomFieldV.J)
F = np.random.random(2)

In [ ]:
# Poisson rhs random û
import mshr
import numpy as np
import matplotlib.pyplot as plt
from helpers import DOMAIN
from helpers_rhs_random import solve_poisson_for_given_sample_rhs_random
from scipy.interpolate import griddata
import plotly.graph_objects as go
from matplotlib.ticker import MaxNLocator


u_sol = solve_poisson_for_given_sample_rhs_random(10, jacobianV, xi, F)

mesh = mshr.generate_mesh(DOMAIN, 10)
x_coords = mesh.coordinates()[:, 0]
y_coords = mesh.coordinates()[:, 1]
grid_x, grid_y = np.mgrid[-1:1:500j, -1:1:500j]
u_sols = []

for i in range(len(x_coords)):
    u_sols.append(u_sol(x_coords[i], y_coords[i]))

grid_z = griddata((x_coords, y_coords), u_sols, (grid_x, grid_y), method='linear')

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
cbar = plt.colorbar(cp)
cbar.ax.tick_params(labelsize=16)
cbar.locator = MaxNLocator(nbins=5)
cbar.update_ticks()
ax.set_xlim([-1.1, 1.1])
ax.set_ylim([-1.1, 1.1])
plt.title(r'$\hat{u}(\hat{x}, \omega)$ on $D_{ref}$', fontsize=24, pad=20)
plt.xlabel(r'$\hat{x}_1$', fontsize=24)
plt.ylabel(r'$\hat{x}_2$', fontsize=24)
ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

print(f"F: {F}")

In [ ]:
# Poisson rhs random u
import mshr
import numpy as np
import matplotlib.pyplot as plt
from helpers import DOMAIN, inverse_mapping
from helpers_rhs_random import solve_poisson_for_given_sample_rhs_random
from scipy.interpolate import griddata
import plotly.graph_objects as go
import fenics as fe
from matplotlib.ticker import MaxNLocator


u_sol = solve_poisson_for_given_sample_rhs_random(10, jacobianV, xi, F)

mesh = mshr.generate_mesh(DOMAIN, 10)
x_coords = mesh.coordinates()[:, 0]
y_coords = mesh.coordinates()[:, 1]
grid_x, grid_y = np.mgrid[-1.5:1.5:500j, -1.5:1.5:500j]
u_sols = []
x_coords_perturbed = []
y_coords_perturbed = []

for i in range(len(x_coords)):
    P_hat = fe.Point(x_coords[i], y_coords[i])
    x_coords_perturbed.append(randomFieldV(P_hat, xi)[0])
    y_coords_perturbed.append(randomFieldV(P_hat, xi)[1])
    u_sols.append(u_sol(P_hat))


grid_z = griddata((x_coords_perturbed, y_coords_perturbed), u_sols, (grid_x, grid_y), method='linear')

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
cp = ax.contourf(grid_x, grid_y, grid_z, levels=100, cmap='viridis')
cbar = plt.colorbar(cp)
cbar.ax.tick_params(labelsize=16)
cbar.locator = MaxNLocator(nbins=5)
cbar.update_ticks()
ax.set_xlim([np.min(x_coords_perturbed) - 0.1, np.max(x_coords_perturbed) + 0.1])
ax.set_ylim([np.min(y_coords_perturbed) - 0.1, np.max(y_coords_perturbed) + 0.1])
plt.title(r'$u(x, \omega)$ on $D(\omega)$', fontsize=24, pad=20)
plt.xlabel(r'$x_1$', fontsize=24)
plt.ylabel(r'$x_2$', fontsize=24)
ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()
print(f"F: {F}")

In [ ]:
# Assuming randomFieldV and jacobianV are already defined
randomFieldV_4, jacobianV_4 = calculate_vector_field_eigenpairs(4)

# used different xi for plot
xi = np.random.uniform(-np.sqrt(6), np.sqrt(6), randomFieldV_4.J)
num_points = 1000
points = []
perturbed_points = []
for i in range(num_points):
    point = np.array([np.cos(2 * np.pi * i / num_points), np.sin(2 * np.pi * i / num_points)])
    points.append(point)
    perturbed_points.append(randomFieldV_4(point, xi))

left_semi_circle = []
right_semi_circle = []
for i in range(num_points):
    if perturbed_points[i][0] < 0:
        if np.linalg.norm(perturbed_points[i]) <= 1:
            left_semi_circle.append(perturbed_points[i])
        else:
            left_semi_circle.append(perturbed_points[i] / np.linalg.norm(perturbed_points[i]))
    else:
        if np.linalg.norm(perturbed_points[i]) <= 1:
            right_semi_circle.append(perturbed_points[i])
        else:
            right_semi_circle.append(perturbed_points[i] / np.linalg.norm(perturbed_points[i]))

# Sort right semi circle points by y coordinate
right_semi_circle = sorted(right_semi_circle, key=lambda x: x[1])

for i in range(num_points):
    left_semi_circle.append(np.array([0, i/num_points]))
    right_semi_circle.append(np.array([0, i/num_points]))
    left_semi_circle.append(np.array([0, - i/num_points]))
    right_semi_circle.append(np.array([0, - i/num_points]))

fig = plt.figure(figsize=(10, 8))

ax = fig.add_subplot(111)
ax.set_aspect('equal', adjustable='datalim')
ax.set_xlim([-1.5, 1.5])
ax.set_ylim([-1.5, 1.5])

# Plot the original circle
# for i in range(num_points):
#     plt.plot([points[i][0], perturbed_points[i][0]], [points[i][1], perturbed_points[i][1]], 'black')

# Convert perturbed points to a numpy array
points = np.array(points)
perturbed_points = np.array(perturbed_points)
left_semi_circle = np.array(left_semi_circle)
right_semi_circle = np.array(right_semi_circle)


# Fill the space inside the perturbed circle
ax.fill(perturbed_points[:, 0], perturbed_points[:, 1], color='lightcoral', alpha=0.4, label=r'$D(\omega_2)$')
# ax.fill(points[:, 0], points[:, 1], color='red', alpha=0.2, label='Unit circle')
ax.fill(left_semi_circle[:, 0], left_semi_circle[:, 1], color='green', alpha=0.2, label=r'$D_{ref} \cap D(\omega_2), x_1 \leq 0$')
ax.fill(right_semi_circle[:, 0], right_semi_circle[:, 1], color='blue', alpha=0.2, label=r'$D_{ref} \cap D(\omega_2), x_1 > 0$')


ax.text(-0.75, 0, r'$\omega_1^{(1)}$', fontsize=24, ha='center', va='center')
ax.text(0.75, 0, r'$\omega_1^{(2)}$', fontsize=24, ha='center', va='center')

plt.xlabel(r'$x_1$', fontsize=24)
plt.ylabel(r'$x_2$', fontsize=24)
# plt.title('Random right hand side f', fontsize=24, pad=20)
ax.tick_params(axis='both', which='major', labelsize=16)
plt.legend(loc='upper left', fontsize=16)

plt.show()


In [ ]:
# plots for poisson mc-analysis

from helpers_rhs_random import *

poisson_rhs_random_analyse_two_resolutions_from_data_u_hat(resolution_sparse = 10,
                                  resolution_fine = 14,
                                  P_hat = fe.Point(0.2, 0.2))

In [ ]:
# Sobol plot

from helpers import calculate_vector_field_eigenpairs
from helpers_rhs_random import poisson_rhs_random_plot_sobols, poisson_rhs_random_sobol_calc_indices_from_data

mesh_res = 8
size_of_xi = 6
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(mesh_res)

S_single, S_total, mc_sample_size = poisson_rhs_random_sobol_calc_indices_from_data(fem_res=mesh_res, kl_res=mesh_res,
                                                                         size_of_xi=size_of_xi,
                                                                         randomFieldV=randomFieldV, jacobianV=jacobianV)
poisson_rhs_random_plot_sobols(S_single, S_total, mc_sample_size)